In [55]:
import numpy as np
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
from pandas import ExcelFile
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import scipy.io as io

# Classification of Asthma dataset using different machine learning tools

We have a datset provided by SARP study, containing spirometery and demographic data for 1358 subjects. The goal of this project is to classify and predict asthma severity using Age of hospital enrolment, Age of Asthma onset, Sex and FEV/FVC ratio of a patient.

## This Section is for data preperation. We load the excel files in a pandas dataframe and explore it.

Creating an Excel file object in pandas

In [2]:
xl = pd.ExcelFile("sarp.xlsx")

Getting the sheet name in the excael file

In [3]:
xl.sheet_names

['sarp']

Reading the information in the excel object in a pandas dataframe

In [37]:
ds = xl.parse('sarp')

Exploring the data

In [7]:
ds.head()

,ID,gender,Age_Enroll,ageasthonset,asthma_status,Baseline_preDrug_FEV1pp,Baseline_preDrug_FVCpp,Baseline_preDrug_FEV1_FVC,status_factorized
0,1,2,22.753425,7,mild,97,137.0,0.622000,1
1,2,1,25.726027,6,mild,94,100.0,0.770197,1
2,3,2,21.893151,11,mild,114,114.0,0.885287,1
3,4,1,39.827397,27,mild,100,105.0,0.761993,1
4,5,1,20.098630,15,mild,80,93.0,0.727473,1


Check if there is any missing values in the columns we need as features

In [35]:
ds.count()

ID                           1358
gender                       1358
Age_Enroll                   1358
ageasthonset                 1358
asthma_status                1358
Baseline_preDrug_FEV1pp      1358
Baseline_preDrug_FVCpp       1324
Baseline_preDrug_FEV1_FVC    1326
status_factorized            1358
dtype: int64

There are missing values in column 'Baseline_preDrug_FEV1_FVC'. We can impute the values or simply just drop them. Here we drop the rows which contain missing values.

In [38]:
ds = ds[['gender', 'Age_Enroll', 'ageasthonset', 'Baseline_preDrug_FEV1_FVC', 'status_factorized']].dropna()

Creating a database only containing the features we need

In [39]:
X = ds[['gender', 'Age_Enroll', 'ageasthonset', 'Baseline_preDrug_FEV1_FVC']]

Our target is the asthma status. The columns 'asthma_status' should be the target, however I have already provided the factorized asthma status column in which, each asthma status is mapped to an integer value (1: mild, 2: moderate, 3: severe)

I created a class to convert the class numbers back to their names.

In [49]:
def show_class(output):
    if output == 1:
        print("Mild asthma")
    elif output == 2:
        print("Moderate asthma")
    else:
        print("Severe asthma")

In [40]:
y = ds['status_factorized']

In [48]:
print(X.count())
print(str(y.name) + '           ' + str(y.count()))

gender                       1326
Age_Enroll                   1326
ageasthonset                 1326
Baseline_preDrug_FEV1_FVC    1326
dtype: int64
status_factorized           1326


Data should be splitted into train and test sets to provide a possibility of evaluating the model on unknown data (test) for which we assume that we don't have the target value. Then we can evaluate the model by comparing the predictions and actual labels (targets).

I set the proportion of test datset to be 0.33 percent of the initial dataset)

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

**Support Vector Machines**
Support vector machine are machine learning tools used for classification. They could be used for classes with nonlinear relationships with the target. They could also work as linear classifiers as well. Before using the SVCs, data shoud be scaled so all the features are in the same range. This is crucial for classifiers that work based on eucleadian distance.

Here I used one linear and one kernel-based SVC. (The kernel I used is the radial basis function kernel)

In [50]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

svm_clf = Pipeline([("scaler", StandardScaler()),
                   ("linear_svc", LinearSVC(C=1, loss="hinge"))])
kernel_svm_clf = Pipeline([("scaler", StandardScaler()), 
                                ("kernel_clf", SVC(kernel="rbf", degree=3, coef0=1, C=5))]) 

Training the linear model

In [53]:
svm_clf.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_svc', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

predicting the results for the test set

In [54]:
y_pred = svm_clf.predict(X_test)

**Evaluation metrics**
We have different evaluation metrics including accuracy, precision, recall, f1_score!

refer to the literature for the definitions but as a summary, the higher the better!

Confusion matrix shows the class divisions and how many of the instances are missclassified. The best matrix is the one with only values on its diagons, meaning that the classification was perfect.

In [71]:
from sklearn.metrics import f1_score 
from sklearn.metrics import confusion_matrix, precision_score, recall_score 

print('Accuracy is: {0}'.format(svm_clf.score(X_test, y_test)))
print('Precision is {0}'.format(precision_score(y_test, y_pred, average='micro')))
print('Recall is {0}'.format(recall_score(y_test, y_pred, average='micro')))
print('f1 score is {0}'.format(f1_score(y_test, y_pred, average='micro')))

Accuracy is: 0.6004566210045662
Precision is 0.6004566210045662
Recall is 0.6004566210045662
f1 score is 0.6004566210045662


In [72]:
confusion_matrix(y_test, y_pred)

array([[134,   0,  37],
       [ 22,   0,  61],
       [ 55,   0, 129]])

Predicting a new target

In [73]:
res = svm_clf.predict([[1, 45, 7, 0.76]]).astype(np.int32)
show_class(res[0])

Mild asthma


The scores and the confusion matrix are not showing satisfactory results. This might be dure to class imbalance, nolinearity or simply mixture of classes.

Let's check the class balances.

In [75]:
y.value_counts()

3    562
1    491
2    273
Name: status_factorized, dtype: int64

We can see that the target values are biased towards mild and severe asthma. We may want to use class weights here to deal with class imbalance. The ratio between classes are mild: 0.37, moderate: 0.21 and severe: 0.42 however we want to put more weight on the lower population classes to avoid bias. Hence I set the class weights argument to 'balanced' to balance the class imbalance. 

In [76]:
svm_clf = Pipeline([("scaler", StandardScaler()),
                   ("linear_svc", LinearSVC(C=1, loss="hinge", class_weight='balanced'))])
kernel_svm_clf = Pipeline([("scaler", StandardScaler()), 
                                ("kernel_clf", SVC(kernel="rbf", degree=3, coef0=1, C=5, class_weight='balanced'))]) 

In [77]:
svm_clf.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_svc', LinearSVC(C=1, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

In [78]:
y_pred = svm_clf.predict(X_test)
print('Accuracy is: {0}'.format(svm_clf.score(X_test, y_test)))
print('Precision is {0}'.format(precision_score(y_test, y_pred, average='micro')))
print('Recall is {0}'.format(recall_score(y_test, y_pred, average='micro')))
print('f1 score is {0}'.format(f1_score(y_test, y_pred, average='micro')))

Accuracy is: 0.6095890410958904
Precision is 0.6095890410958904
Recall is 0.6095890410958904
f1 score is 0.6095890410958904


In [79]:
confusion_matrix(y_test, y_pred)

array([[139,   0,  32],
       [ 24,   0,  59],
       [ 56,   0, 128]])

It did not work much. Lets try a nonlinear model

In [80]:
kernel_svm_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('kernel_clf', SVC(C=5, cache_size=200, class_weight='balanced', coef0=1,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [81]:
y_pred = svm_clf.predict(X_test)
print('Accuracy is: {0}'.format(svm_clf.score(X_test, y_test)))
print('Precision is {0}'.format(precision_score(y_test, y_pred, average='micro')))
print('Recall is {0}'.format(recall_score(y_test, y_pred, average='micro')))
print('f1 score is {0}'.format(f1_score(y_test, y_pred, average='micro')))

Accuracy is: 0.6095890410958904
Precision is 0.6095890410958904
Recall is 0.6095890410958904
f1 score is 0.6095890410958904


In [82]:
confusion_matrix(y_test, y_pred)

array([[139,   0,  32],
       [ 24,   0,  59],
       [ 56,   0, 128]])

**K nearest neighbors**

Now we use different classifier. K nearest neighbors which works based on the k nearest points to the target to evaluate the target class.

In [100]:
from sklearn.neighbors import KNeighborsClassifier as knc
knn_clf = Pipeline([("scaler", StandardScaler()),
                    ("knc", knc(weights='distance', n_jobs=-1, n_neighbors=10))])

In [101]:
knn_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knc', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=10, p=2,
           weights='distance'))])

In [102]:
y_pred = knn_clf.predict(X_test)
print('Accuracy is: {0}'.format(svm_clf.score(X_test, y_test)))
print('Precision is {0}'.format(precision_score(y_test, y_pred, average='micro')))
print('Recall is {0}'.format(recall_score(y_test, y_pred, average='micro')))
print('f1 score is {0}'.format(f1_score(y_test, y_pred, average='micro')))

Accuracy is: 0.6095890410958904
Precision is 0.634703196347032
Recall is 0.634703196347032
f1 score is 0.634703196347032


In [103]:
confusion_matrix(y_test, y_pred)

array([[120,  14,  37],
       [ 21,  24,  38],
       [ 33,  17, 134]])

In [104]:
res = knn_clf.predict([[1, 45, 7, 0.76]]).astype(np.int32)
show_class(res[0])

Moderate asthma


It did better for moderate classes but got worse for the other two. How could we know what number of neighbors we should select. Choosing hyperparameters could be a matter of trial and error. However, there is a gridsearch feature we can use to automate it. Let's do it for the number of neighbors in the Knn classifier.

cv argument below is for the cross-validation (It estimates the performance of the model by splitting the dataset into K (here 5) folds and takes one of them as unknown target at a time))

In [112]:
from sklearn.model_selection import GridSearchCV as gs

param_grid = [ {'knc__n_neighbors': [3, 5, 7, 9, 11, 13]} ]
grid_search = gs(knn_clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knc', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=10, p=2,
           weights='distance'))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'knc__n_neighbors': [3, 5, 7, 9, 11, 13]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [113]:
grid_search.best_params_

{'knc__n_neighbors': 13}

In [114]:
y_pred = grid_search.predict(X_test)
print('Accuracy is: {0}'.format(svm_clf.score(X_test, y_test)))
print('Precision is {0}'.format(precision_score(y_test, y_pred, average='micro')))
print('Recall is {0}'.format(recall_score(y_test, y_pred, average='micro')))
print('f1 score is {0}'.format(f1_score(y_test, y_pred, average='micro')))

Accuracy is: 0.6095890410958904
Precision is 0.634703196347032
Recall is 0.634703196347032
f1 score is 0.634703196347032


In [115]:
confusion_matrix(y_test, y_pred)

array([[119,  14,  38],
       [ 21,  20,  42],
       [ 32,  13, 139]])

In [118]:
res = grid_search.predict([[1, 45, 7, 0.76]]).astype(np.int32)
show_class(res[0])

Moderate asthma


It improved the results slightly. 

Let's try a decision tree classifier.

**Decision trees**

In [116]:
from sklearn.tree import DecisionTreeClassifier 

dt_clf = DecisionTreeClassifier()

In [117]:
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [119]:
y_pred = dt_clf.predict(X_test)
print('Accuracy is: {0}'.format(svm_clf.score(X_test, y_test)))
print('Precision is {0}'.format(precision_score(y_test, y_pred, average='micro')))
print('Recall is {0}'.format(recall_score(y_test, y_pred, average='micro')))
print('f1 score is {0}'.format(f1_score(y_test, y_pred, average='micro')))

Accuracy is: 0.6095890410958904
Precision is 0.5365296803652968
Recall is 0.5365296803652968
f1 score is 0.5365296803652968


In [120]:
confusion_matrix(y_test, y_pred)

array([[106,  19,  46],
       [ 26,  25,  32],
       [ 28,  52, 104]])

It doesn't seem that it is improving the results. What if we combine many of these trees and decide beased on the class that gets the highest votes from the trees. This could be done using random forests.

**Random forests**

In [121]:
from sklearn.ensemble import RandomForestClassifier 
rfc_clf = RandomForestClassifier( n_estimators = 500, n_jobs =-1) 

In [123]:
y_pred = dt_clf.predict(X_test)
print('Accuracy is: {0}'.format(svm_clf.score(X_test, y_test)))
print('Precision is {0}'.format(precision_score(y_test, y_pred, average='micro')))
print('Recall is {0}'.format(recall_score(y_test, y_pred, average='micro')))
print('f1 score is {0}'.format(f1_score(y_test, y_pred, average='micro')))

Accuracy is: 0.6095890410958904
Precision is 0.5365296803652968
Recall is 0.5365296803652968
f1 score is 0.5365296803652968


In [122]:
rfc_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [124]:
confusion_matrix(y_test, y_pred)

array([[106,  19,  46],
       [ 26,  25,  32],
       [ 28,  52, 104]])